In [23]:
import pandas as pd
import csv
import re
import numpy as np
import os
import random
from gensim.models import word2vec
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import sys

In [24]:
vector_num = 20
size = 100

In [25]:
df_train = pd.read_csv('./train.csv')  
#print(df_train)  

In [26]:
df_test = pd.read_csv('./test.csv')  
#print(df_test) 

In [27]:
#train data
train_data = []
train_label = []
for i in range(0,len(df_train)):
    train_data.append(df_train.iloc[i,1])
    train_label.append(df_train.iloc[i,3])
#print(len(train_data))
#print(len(train_label))
train_label = np.array(train_label)
train_label = np.reshape(train_label,(2040,1))
#print(train_label)

In [28]:
#test data
test_data = []
for i in range(0,len(df_test)):
    test_data.append(df_test.iloc[i,1])
#print((test_data))

In [29]:
#以空格切句子為單字
#刪去一些標點符號
train_seq=[]
test_seq = []
for i in train_data:
    a1 = re.sub('[?]', '', i)
    a2 = re.sub('[.]', '', a1)
    a3 = re.sub('[;]', '', a2)
    a4 = re.sub('[:]', '', a3)
    a5 = re.sub('[-]', '', a4)
    a6 = re.sub('[!]', '', a5)
    a7 = re.sub('[(]', '', a6)
    a8 = re.sub('[)]', '', a7)
    a = re.sub('[,]', '', a8)
    #print(i)
    a = a.split()
    #temp.append(a)
    train_seq.append(a)
#print((train_seq[0][3]))

for i in test_data:
    a1 = re.sub('[?]', '', i)
    a2 = re.sub('[.]', '', a1)
    a3 = re.sub('[;]', '', a2)
    a4 = re.sub('[:]', '', a3)
    a5 = re.sub('[-]', '', a4)
    a6 = re.sub('[!]', '', a5)
    a7 = re.sub('[(]', '', a6)
    a8 = re.sub('[)]', '', a7)
    a = re.sub('[,]', '', a8)
    #print(i)
    a = a.split()
    test_seq.append(a)

#print((test_seq))

In [30]:
f = open('train_seq.txt','a+')
for i in train_seq:
    for j in i:
        print(j, file = f)

In [31]:
f = open('test_seq.txt','a+')
for i in test_seq:
    for j in i:
        print(j, file = f)

In [32]:
sentences = word2vec.Text8Corpus('./train_seq.txt')
model_train = word2vec.Word2Vec(sentences, sg=1, size=size,  window=5,  min_count=5,  negative=3, sample=0.001, hs=1, workers=4)
#model.save('./train_model.txt')
#print(model)

In [33]:
sentences_test = word2vec.Text8Corpus('./test_seq.txt')
model_test = word2vec.Word2Vec(sentences_test, sg=1, size=size,  window=5,  min_count=5,  negative=3, sample=0.001, hs=1, workers=4)


In [34]:
train_vector = []
for sentence in train_seq:
    #print(sentence)
    temp = []
    for vocab in sentence:
        if vocab.lower() in model_train:
            a = model_train[vocab.lower()]
            temp.append(a)
        elif vocab.lower() not in model_train:
            continue
    train_vector.append(temp)
    #print(len(temp))
#print((train_vector[0][5][34]))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if __name__ == '__main__':


In [35]:
test_vector = []
for sentence in test_seq:
    #print(sentence)
    temp = []
    for vocab in sentence:
        if vocab.lower() in model_test:
            a = model_test[vocab.lower()]
            #print(a)
            temp.append(a)
        elif vocab.lower() not in model_test:
            temp.append(np.zeros((size)))
            continue
    test_vector.append(temp)
"""for i in range(0,len(test_vector)):    
    print(len(test_vector[i]))"""

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


'for i in range(0,len(test_vector)):    \n    print(len(test_vector[i]))'

In [36]:
#選擇每個句子取的vector數
train_vector_final = []
test_vector_final = []
for i in train_vector:
    #print(i)
    if len(i) > vector_num:
        train_vector_final.append(i[0:vector_num])
    elif len(i) < vector_num:
        a = np.vstack((i, np.zeros((vector_num-len(i),size))))
        #print(len(a))
        train_vector_final.append(a)
    else:
        train_vector_final.append(i[0:vector_num])
#print((train_vector_final[0:10]))


for i in test_vector:
    #print(len(test_vector))
    #print(len(i[6]))
    if len(i) > vector_num:
        test_vector_final.append(i[0:vector_num])
    elif len(i) < vector_num:
        #print(len(i),vector_num - len(i))
        #print(i)
        #print(np.zeros((vector_num-len(i),size)))
        b = np.vstack((i,np.zeros((vector_num-len(i),size))))
        #print(len(a))
        test_vector_final.append(b)
    else:
        test_vector_final.append(i[0:vector_num])
#print(len(test_vector_final[7]))

train_vector_final = np.array(train_vector_final)

#train_vector_final = np.reshape(train_vector_final,(2040,15))
test_vector_final = np.array(test_vector_final)

In [37]:
#lstm 開始

def buildManyToOneModel(shape):
    model = Sequential()
    model.add(LSTM(size ,input_length = vector_num ,input_dim = size))
    model.add(Dropout(0.5))
    # output shape: (1, 1)
    model.add(Dense(10))
    model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model

In [38]:
def shuffle(X,Y):
    np.random.seed(10)
    X = np.array(X)
    Y = np.array(Y)
    randomList = np.arange(X.shape[0])
    np.random.shuffle(randomList)
    return X[randomList], Y[randomList]

In [39]:
def splitData(X,Y,rate):
    X_train = X[int(X.shape[0]*rate):]
    Y_train = Y[int(Y.shape[0]*rate):]
    X_val = X[:int(X.shape[0]*rate)]
    Y_val = Y[:int(Y.shape[0]*rate)]
    return X_train, Y_train, X_val, Y_val

In [ ]:
X_train, Y_train = train_vector_final,train_label
X_train, Y_train = shuffle(X_train, Y_train)
# because no return sequence, Y_train and Y_val shape must be 2 dimension
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)
#print(len(X_train))
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_val = np.array(X_val)
Y_val = np.array(Y_val)
model = buildManyToOneModel(X_train)
#model.add(Dropout(0.2))
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=7, batch_size=1, validation_data=(X_val, Y_val), callbacks=[callback])


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 81,421
Trainable params: 81,421
Non-trainable params: 0
_________________________________________________________________
Epoch 1/7
1836/1836 [==============================] - 10s 5ms/step - loss: 0.7973 - val_loss: 0.5268
Epoch 2/7
1053/1836 [================>.............] - ETA: 3s - loss: 0.7043

In [ ]:
model.save('309513121_model.h5') 

In [ ]:
x_test = test_vector_final
result = model.predict(x_test)
print(result)


In [ ]:
with open('309513121_predict.csv', 'w', newline='') as csvfile:
  # 建立 CSV 檔寫入器
    writer = csv.writer(csvfile)
    writer.writerow(['ID','Label'])
    for i in range(0,len(result)):
        a1 = re.sub('[[]', '', str(result[i]))
        a2 = re.sub('[]]', '', a1)
        writer = csv.writer(csvfile)
        writer.writerow([i+1,a2])


In [ ]:
a = sys.argv[0]